# Remote tensors

In [1]:
import syft as sy
import torch as th


In [2]:
hook = sy.TorchHook(th)


In [3]:
viper = sy.VirtualWorker(hook, id = "viper")
quassi = sy.VirtualWorker(hook, id = "quassi")

In [4]:
x = th.tensor([1, 2, 3, 4, 5]).send(viper)
y = th.tensor([2, 2, 2, 2, 2]).send(quassi)

In [5]:
z = x + y #raises error because x and y are not on same machine

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:viper #objects:1> while the other is on <VirtualWorker id:quassi #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

In [6]:
x = th.tensor([1, 2, 3, 4, 5]).send(viper)
y = th.tensor([2, 2, 2, 2, 2]).send(viper)

In [7]:
z = x + y #now treats as the individaul tensor

In [8]:
z.get()

tensor([3, 4, 5, 6, 7])

In [9]:
z = th.add(x ,y) #same as x + y
z

(Wrapper)>[PointerTensor | me:20353557435 -> viper:94648250825]

In [10]:
z.get()

tensor([3, 4, 5, 6, 7])

# Simple linear model

In [11]:
inputs = th.tensor([[0., 1], [0, 0], [1, 1], [1, 0]], requires_grad = True).send(viper)
target = th.tensor([[1.], [0], [1], [0]], requires_grad = True).send(viper)

In [12]:
weights = th.tensor([[0.], [0]], requires_grad = True).send(viper)

In [13]:
for i in range(10):

    pred = inputs.mm(weights)

    loss = ((pred - target)**2).sum() #mse

    loss.backward() #backporpagate

    weights.data.sub_(weights.grad * 0.1) #wt update
    weights.grad *= 0

    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Day 14 from here

In [14]:
 #The worker viper has 6 objects 
print(len(viper._objects), viper._objects)

6 {16332240421: tensor([1, 2, 3, 4, 5]), 59865927080: tensor([2, 2, 2, 2, 2]), 66991115501: tensor([[0., 1.],
        [0., 0.],
        [1., 1.],
        [1., 0.]], requires_grad=True), 68694660122: tensor([[1.],
        [0.],
        [1.],
        [0.]], requires_grad=True), 30100629494: tensor([[0.0536],
        [0.9463]], requires_grad=True), 71198641411: tensor([[0.9328],
        [0.0000],
        [0.9997],
        [0.0670]], grad_fn=<MmBackward>)}


In [15]:
viper = viper.clear_objects() #clear the objects bounded to viper

In [16]:
viper._objects

{}

In [17]:
#pysfyt allows us to manage life cycle of pointer ex:
x = th.tensor([2, 2, 3, 5, 8])
x.send(viper)

(Wrapper)>[PointerTensor | me:43716112630 -> viper:56376786573]

In [18]:
del x

In [19]:
viper._objects

{56376786573: tensor([2, 2, 3, 5, 8])}

In [34]:
x = th.tensor([2, 2, 6, 5, 8]).send(quassi)
y = th.tensor([3]).send(viper)

In [30]:
#check garbage collector
x.child.garbage_collect_data

True

In [38]:
quassi._objects

{}

In [36]:
x = "vipersfasfsadf"

In [39]:
quassi._objects #now quassi automatically loses the pointer to x

{}

In [41]:
Out

AttributeError: 'Tensor' object has no attribute 'child'

In [49]:
for i in range(1000):
    x = th.tensor([2, 4, 6 ,8]).send(quassi) # in every loop, reassigned x and deleted the previous pointer for garbage collection
    

In [50]:
quassi._objects

{72124152416: tensor([2, 4, 6, 8]), 72540281812: tensor([2, 4, 6, 8])}

##### No matter how many times we send the tensor on quassi, it will be reassigned and the previous will be deleted. So we have less mamory usage. 


## Common errors:
1. > Using tensors from different machines 
2. > Using tensors and pointers on operation

In [53]:
#1. 
x = th.tensor([4]).send(viper)
y = th.tensor([5]).send(quassi)

z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:viper #objects:3> while the other is on <VirtualWorker id:quassi #objects:2>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

In [54]:
#2. 
x = th.tensor([6]).send(viper)
y = th.tensor([4])

z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually located on another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(viper) on the other tensor.

Tensor A: [PointerTensor | me:35995789271 -> viper:94828737468]
Tensor B: tensor([4])